[View in Colaboratory](https://colab.research.google.com/github/david7joy/RNNProjects/blob/master/Baby_Name_Generator.ipynb)

In [16]:
ls /content/drive/ColabNotebooks/ProjectRNN/

Baby Name Generator.ipynb  babynames.txt  dinos.txt  logs/  Untitled0.ipynb


In [67]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
tf.reset_default_graph()

In [0]:
import numpy as np
from tensorflow.contrib import rnn
import random
import os
import time

In [0]:
logs_path = '/content/drive/ColabNotebooks/ProjectRNN/logs/'
writer = tf.summary.FileWriter(logs_path)

In [169]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
ls /drive

ls: cannot access '/drive': No such file or directory


In [0]:
os.chdir('/content/drive/ColabNotebooks/ProjectRNN/')

In [201]:
file = os.listdir()
file = file[2]
file

'dinos.txt'

In [0]:
with open(file) as f:
    training_data = f.read()
    training_data = training_data.replace('\n',' ')
    training_data = training_data.lower()

In [0]:
chars = list(set(training_data))

In [204]:
data_size,vocab_size = len(training_data),len(chars)
print("The size of the data is %d and vocab size %d"%(data_size,vocab_size))

The size of the data is 19909 and vocab size 27


In [0]:
char_to_ix = {ch:i for i,ch in enumerate(sorted(chars))}
ix_to_char = {i:ch for i,ch in enumerate(sorted(chars))}

In [0]:
# no of hidden cells or units in rnn
n_hidden = 128

# hyperparameters 

learning_rate = .001
training_iters = 75000
display_step = 5000
time_steps = 3
num_input = 1
batch_size = 600

In [0]:
x = tf.placeholder("float",[batch_size,time_steps,num_input])  # the size of each input is 1*19
y = tf.placeholder("float",[batch_size,vocab_size])

In [0]:
weights = {'out':tf.Variable(tf.random_normal([n_hidden,vocab_size]))}
biases = {'out':tf.Variable(tf.random_normal([vocab_size]))}

In [0]:
def RNN(x,weights,biases):
    x = tf.reshape(x,[-1,time_steps])
    x = tf.split(x,time_steps,1)
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    #rnn_cell = rnn.BasicLSTMCell(n_hidden)
    outputs,states = rnn.static_rnn(rnn_cell,x,dtype=tf.float32)
    check1,check2 = outputs,states
    return tf.matmul(outputs[-1],weights['out']) + biases['out']

In [0]:
pred = RNN(x,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.9,beta2=0.999,epsilon=1e-08)
grads_and_vars = optimizer.compute_gradients(cost)
capped_gvs = [(tf.clip_by_value(grad, -1, 1), var) for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

In [0]:
# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [215]:
#initializing the variable

init = tf.global_variables_initializer()

start_time = time.time()
with tf.Session() as sess:
    sess.run(init)
    total_loss = 0
    total_acc  = 0
    offset = 0
    
    #saver = tf.train.Saver()
    for steps in range(1, training_iters + 1):
        if offset > (len(training_data) - 1000):
            offset = random.randint(0,time_steps)

        # getting the input sequence
        print("steps :%d offset1: %d range: %d" %(steps,offset,batch_size*time_steps))
       
        X = [char_to_ix[str(training_data[i])] for i in range(offset, offset + (batch_size*time_steps))]
        X = np.reshape(np.array(X),[batch_size,time_steps,1])
        batch_size = X.shape[0]

        # getting ready the True Y
        zeros_array = np.zeros((batch_size,vocab_size), dtype=float)
        for i in range(batch_size):
            zeros_array[i][char_to_ix[str(training_data[offset + time_steps])]] = 1.0
            offset += time_steps
            
        Y = np.reshape(zeros_array, [batch_size, -1])
        
        #_, acc, cost,one_hot_pred = sess.run([optimizer, accuracy, cost, pred], feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        _, acc, loss, onehot_pred = sess.run([training_op, accuracy, cost, pred], feed_dict={x: X, y: Y})
        total_loss += loss
        total_acc += acc
        if steps % display_step == 0:
            print(steps,(100*total_acc)/steps,total_loss/steps)
            

    # Save the variables to disk.
    #save_path = saver.save(sess, "/Users/davidjoy/Desktop/Github:Study Material/6341 - Applied Machine Learning/Rnn/Dinosaur Project")
    #print("Model saved in path: %s" % save_path)
    
    #sampling
    print("Total Mins {}".format((time.time()-start_time)/60))
    while True:
      inputwords = "enter 3 letters " 
      enteredletters = input(inputwords)
      letters = enteredletters.strip()
      letters = list(letters)
      if len(letters) != time_steps:
            continue
      try:
        letters = [char_to_ix[i] for i in letters]
        for i in range(10):
          fixing_shape = np.zeros((batch_size,3,1), dtype=float)
          cell_input = np.reshape(np.array(letters), [-1, time_steps, 1])
          cell_input = cell_input+fixing_shape
          pred_letter = sess.run(pred,feed_dict={x:cell_input})
          pred_letter = np.reshape(np.array(pred_letter[1]),(1,vocab_size))
          pred_location = int(tf.argmax(pred_letter,axis=1).eval())
          pred_location
          enteredletters += "%s"%(ix_to_char[pred_location])
          letters.append(pred_location)
          letters = letters[1:]
          names = enteredletters.split()
          for i in names:
            print(i)
      except:
        print("letter not in dictionary")



steps :1 offset1: 0 range: 1800
steps :2 offset1: 1800 range: 1800
steps :3 offset1: 3600 range: 1800
steps :4 offset1: 5400 range: 1800
steps :5 offset1: 7200 range: 1800
steps :6 offset1: 9000 range: 1800
steps :7 offset1: 10800 range: 1800
steps :8 offset1: 12600 range: 1800
steps :9 offset1: 14400 range: 1800
steps :10 offset1: 16200 range: 1800
steps :11 offset1: 18000 range: 1800
steps :12 offset1: 1 range: 1800
steps :13 offset1: 1801 range: 1800
steps :14 offset1: 3601 range: 1800
steps :15 offset1: 5401 range: 1800
steps :16 offset1: 7201 range: 1800
steps :17 offset1: 9001 range: 1800
steps :18 offset1: 10801 range: 1800
steps :19 offset1: 12601 range: 1800
steps :20 offset1: 14401 range: 1800
steps :21 offset1: 16201 range: 1800
steps :22 offset1: 18001 range: 1800
steps :23 offset1: 1 range: 1800
steps :24 offset1: 1801 range: 1800
steps :25 offset1: 3601 range: 1800
steps :26 offset1: 5401 range: 1800
steps :27 offset1: 7201 range: 1800
steps :28 offset1: 9001 range: 1800


steps :289 offset1: 3600 range: 1800
steps :290 offset1: 5400 range: 1800
steps :291 offset1: 7200 range: 1800
steps :292 offset1: 9000 range: 1800
steps :293 offset1: 10800 range: 1800
steps :294 offset1: 12600 range: 1800
steps :295 offset1: 14400 range: 1800
steps :296 offset1: 16200 range: 1800
steps :297 offset1: 18000 range: 1800
steps :298 offset1: 0 range: 1800
steps :299 offset1: 1800 range: 1800
steps :300 offset1: 3600 range: 1800
steps :301 offset1: 5400 range: 1800
steps :302 offset1: 7200 range: 1800
steps :303 offset1: 9000 range: 1800
steps :304 offset1: 10800 range: 1800
steps :305 offset1: 12600 range: 1800
steps :306 offset1: 14400 range: 1800
steps :307 offset1: 16200 range: 1800
steps :308 offset1: 18000 range: 1800
steps :309 offset1: 0 range: 1800
steps :310 offset1: 1800 range: 1800
steps :311 offset1: 3600 range: 1800
steps :312 offset1: 5400 range: 1800
steps :313 offset1: 7200 range: 1800
steps :314 offset1: 9000 range: 1800
steps :315 offset1: 10800 range: 1

steps :575 offset1: 3600 range: 1800
steps :576 offset1: 5400 range: 1800
steps :577 offset1: 7200 range: 1800
steps :578 offset1: 9000 range: 1800
steps :579 offset1: 10800 range: 1800
steps :580 offset1: 12600 range: 1800
steps :581 offset1: 14400 range: 1800
steps :582 offset1: 16200 range: 1800
steps :583 offset1: 18000 range: 1800
steps :584 offset1: 0 range: 1800
steps :585 offset1: 1800 range: 1800
steps :586 offset1: 3600 range: 1800
steps :587 offset1: 5400 range: 1800
steps :588 offset1: 7200 range: 1800
steps :589 offset1: 9000 range: 1800
steps :590 offset1: 10800 range: 1800
steps :591 offset1: 12600 range: 1800
steps :592 offset1: 14400 range: 1800
steps :593 offset1: 16200 range: 1800
steps :594 offset1: 18000 range: 1800
steps :595 offset1: 2 range: 1800
steps :596 offset1: 1802 range: 1800
steps :597 offset1: 3602 range: 1800
steps :598 offset1: 5402 range: 1800
steps :599 offset1: 7202 range: 1800
steps :600 offset1: 9002 range: 1800
steps :601 offset1: 10802 range: 1

steps :859 offset1: 1 range: 1800
steps :860 offset1: 1801 range: 1800
steps :861 offset1: 3601 range: 1800
steps :862 offset1: 5401 range: 1800
steps :863 offset1: 7201 range: 1800
steps :864 offset1: 9001 range: 1800
steps :865 offset1: 10801 range: 1800
steps :866 offset1: 12601 range: 1800
steps :867 offset1: 14401 range: 1800
steps :868 offset1: 16201 range: 1800
steps :869 offset1: 18001 range: 1800
steps :870 offset1: 1 range: 1800
steps :871 offset1: 1801 range: 1800
steps :872 offset1: 3601 range: 1800
steps :873 offset1: 5401 range: 1800
steps :874 offset1: 7201 range: 1800
steps :875 offset1: 9001 range: 1800
steps :876 offset1: 10801 range: 1800
steps :877 offset1: 12601 range: 1800
steps :878 offset1: 14401 range: 1800
steps :879 offset1: 16201 range: 1800
steps :880 offset1: 18001 range: 1800
steps :881 offset1: 2 range: 1800
steps :882 offset1: 1802 range: 1800
steps :883 offset1: 3602 range: 1800
steps :884 offset1: 5402 range: 1800
steps :885 offset1: 7202 range: 1800


steps :1144 offset1: 18001 range: 1800
steps :1145 offset1: 1 range: 1800
steps :1146 offset1: 1801 range: 1800
steps :1147 offset1: 3601 range: 1800
steps :1148 offset1: 5401 range: 1800
steps :1149 offset1: 7201 range: 1800
steps :1150 offset1: 9001 range: 1800
steps :1151 offset1: 10801 range: 1800
steps :1152 offset1: 12601 range: 1800
steps :1153 offset1: 14401 range: 1800
steps :1154 offset1: 16201 range: 1800
steps :1155 offset1: 18001 range: 1800
steps :1156 offset1: 0 range: 1800
steps :1157 offset1: 1800 range: 1800
steps :1158 offset1: 3600 range: 1800
steps :1159 offset1: 5400 range: 1800
steps :1160 offset1: 7200 range: 1800
steps :1161 offset1: 9000 range: 1800
steps :1162 offset1: 10800 range: 1800
steps :1163 offset1: 12600 range: 1800
steps :1164 offset1: 14400 range: 1800
steps :1165 offset1: 16200 range: 1800
steps :1166 offset1: 18000 range: 1800
steps :1167 offset1: 0 range: 1800
steps :1168 offset1: 1800 range: 1800
steps :1169 offset1: 3600 range: 1800
steps :117

...

steps :2003 offset1: 0 range: 1800
steps :2004 offset1: 1800 range: 1800
steps :2005 offset1: 3600 range: 1800
steps :2006 offset1: 5400 range: 1800
steps :2007 offset1: 7200 range: 1800
steps :2008 offset1: 9000 range: 1800
steps :2009 offset1: 10800 range: 1800
steps :2010 offset1: 12600 range: 1800
steps :2011 offset1: 14400 range: 1800
steps :2012 offset1: 16200 range: 1800
steps :2013 offset1: 18000 range: 1800
steps :2014 offset1: 0 range: 1800
steps :2015 offset1: 1800 range: 1800
steps :2016 offset1: 3600 range: 1800
steps :2017 offset1: 5400 range: 1800
steps :2018 offset1: 7200 range: 1800
steps :2019 offset1: 9000 range: 1800
steps :2020 offset1: 10800 range: 1800
steps :2021 offset1: 12600 range: 1800
steps :2022 offset1: 14400 range: 1800
steps :2023 offset1: 16200 range: 1800
steps :2024 offset1: 18000 range: 1800
steps :2025 offset1: 1 range: 1800
steps :2026 offset1: 1801 range: 1800
steps :2027 offset1: 3601 range: 1800
steps :2028 offset1: 5401 range: 1800
steps :2029

steps :2291 offset1: 3603 range: 1800
steps :2292 offset1: 5403 range: 1800
steps :2293 offset1: 7203 range: 1800
steps :2294 offset1: 9003 range: 1800
steps :2295 offset1: 10803 range: 1800
steps :2296 offset1: 12603 range: 1800
steps :2297 offset1: 14403 range: 1800
steps :2298 offset1: 16203 range: 1800
steps :2299 offset1: 18003 range: 1800
steps :2300 offset1: 2 range: 1800
steps :2301 offset1: 1802 range: 1800
steps :2302 offset1: 3602 range: 1800
steps :2303 offset1: 5402 range: 1800
steps :2304 offset1: 7202 range: 1800
steps :2305 offset1: 9002 range: 1800
steps :2306 offset1: 10802 range: 1800
steps :2307 offset1: 12602 range: 1800
steps :2308 offset1: 14402 range: 1800
steps :2309 offset1: 16202 range: 1800
steps :2310 offset1: 18002 range: 1800
steps :2311 offset1: 2 range: 1800
steps :2312 offset1: 1802 range: 1800
steps :2313 offset1: 3602 range: 1800
steps :2314 offset1: 5402 range: 1800
steps :2315 offset1: 7202 range: 1800
steps :2316 offset1: 9002 range: 1800
steps :2

steps :2578 offset1: 5403 range: 1800
steps :2579 offset1: 7203 range: 1800
steps :2580 offset1: 9003 range: 1800
steps :2581 offset1: 10803 range: 1800
steps :2582 offset1: 12603 range: 1800
steps :2583 offset1: 14403 range: 1800
steps :2584 offset1: 16203 range: 1800
steps :2585 offset1: 18003 range: 1800
steps :2586 offset1: 3 range: 1800
steps :2587 offset1: 1803 range: 1800
steps :2588 offset1: 3603 range: 1800
steps :2589 offset1: 5403 range: 1800
steps :2590 offset1: 7203 range: 1800
steps :2591 offset1: 9003 range: 1800
steps :2592 offset1: 10803 range: 1800
steps :2593 offset1: 12603 range: 1800
steps :2594 offset1: 14403 range: 1800
steps :2595 offset1: 16203 range: 1800
steps :2596 offset1: 18003 range: 1800
steps :2597 offset1: 3 range: 1800
steps :2598 offset1: 1803 range: 1800
steps :2599 offset1: 3603 range: 1800
steps :2600 offset1: 5403 range: 1800
steps :2601 offset1: 7203 range: 1800
steps :2602 offset1: 9003 range: 1800
steps :2603 offset1: 10803 range: 1800
steps :

steps :2867 offset1: 10801 range: 1800
steps :2868 offset1: 12601 range: 1800
steps :2869 offset1: 14401 range: 1800
steps :2870 offset1: 16201 range: 1800
steps :2871 offset1: 18001 range: 1800
steps :2872 offset1: 1 range: 1800
steps :2873 offset1: 1801 range: 1800
steps :2874 offset1: 3601 range: 1800
steps :2875 offset1: 5401 range: 1800
steps :2876 offset1: 7201 range: 1800
steps :2877 offset1: 9001 range: 1800
steps :2878 offset1: 10801 range: 1800
steps :2879 offset1: 12601 range: 1800
steps :2880 offset1: 14401 range: 1800
steps :2881 offset1: 16201 range: 1800
steps :2882 offset1: 18001 range: 1800
steps :2883 offset1: 2 range: 1800
steps :2884 offset1: 1802 range: 1800
steps :2885 offset1: 3602 range: 1800
steps :2886 offset1: 5402 range: 1800
steps :2887 offset1: 7202 range: 1800
steps :2888 offset1: 9002 range: 1800
steps :2889 offset1: 10802 range: 1800
steps :2890 offset1: 12602 range: 1800
steps :2891 offset1: 14402 range: 1800
steps :2892 offset1: 16202 range: 1800
step

KeyboardInterrupt: ignored

In [0]:
offset

19200